# Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.38'

In [7]:
import os
DayFolder=os.getcwd()
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [8]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [9]:
df2

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,1mol_FIA_R1,102.755387,1286.320661,2541.431492,1286.320661,5434.007114,0.856467,14.492672,4.567590e-01,...,0.537666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1mol_FIA_R2,102.704835,1286.472665,2178.012056,1286.472715,5146.514377,0.966401,15.566447,4.016323e-01,...,1.508602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1mol_FIA_R3,102.738491,1286.539466,1955.070395,1286.539466,4873.005537,1.009231,22.927820,4.280530e-01,...,0.539208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1mol_FIB_R1,102.740812,1285.145501,2042.930606,1285.145551,5320.261401,1.052424,13.460766,4.333486e-01,...,0.810475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1mol_FIB_R2,102.794406,1286.110226,4424.605047,1286.110226,6668.735954,0.550271,27.653737,6.909900e-01,...,0.405193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,1mol_FIB_R3,102.747669,1286.319781,1187.233603,1286.319781,2648.671539,0.891045,19.598101,4.644300e-01,...,1.219329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,4mol_FID_R1,102.914929,1286.324081,3609.910911,1286.324131,9204.231781,0.930378,25.216835,6.930026e-01,...,0.387153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,4mol_FID_R2,102.881307,1286.211916,7593.931107,1286.211966,16472.246827,0.804433,18.348584,6.532680e-01,...,1.010533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,4mol_FID_R3,102.892689,1286.317689,4102.195360,1286.317739,10500.866592,0.948736,18.204414,6.549753e-01,...,0.963180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,10mol_FID_R1,102.869562,1285.522186,5556.129322,1285.522236,15613.030641,1.112515,26.011465,4.882108e-01,...,1.038884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [14]:
filetype='headless_txt' # Specifies raman file format, here a text file with no header
file_ext='.txt' # File extension, a txt file
exclude_str=['Ne', 'NE', 'Si', 'nodiad', 'AutoCalibration '] 
prefix=False # File doesn't have a 01, 02 prefix, as WItec can add
# Put the common string you can in all your Ne lines here
Diad_Files=pf.get_diad_files(path=spectra_path, 
                             file_ext=file_ext, exclude_str=exclude_str)

meta=pf.loop_convert_datastamp_to_metadata(path=spectra_path, 
files=Diad_Files, creation=False,
modification=True)

## Then get a simple file name you can stitch with the spectra

In [15]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

'10mol_FIE_R1'

## Now get filename from spectra into same form as metadata

In [16]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s

array(['1mol_FIA_R1', '1mol_FIA_R2', '1mol_FIA_R3', '1mol_FIB_R1',
       '1mol_FIB_R2', '1mol_FIB_R3', '4mol_FID_R1', '4mol_FID_R2',
       '4mol_FID_R3', '10mol_FID_R1', '10mol_FID_R1_FIN', '10mol_FID_R2',
       '10mol_FID_R2_FIN', '10mol_FID_R3', '10mol_FID_R3_FIN',
       '10mol_FIE_R1', '10mol_FIE_R1_FIN', '10mol_FIE_R2',
       '10mol_FIE_R2_FIN', '10mol_FIE_R3', '10mol_FIE_R3_FIN',
       'LL8_404a_r1', 'LL8_404a_r2', 'LL8_404a_r3', 'LL8_612_a_r1',
       'LL8_612_a_r2', 'LL8_612_a_r3', 'LL8_612_c_r1', 'LL8_612_c_r2',
       'LL8_612_c_r3', 'LL8_615_r1', 'LL8_615_r2', 'LL8_615_r3',
       'LL8_622B_R1', 'LL8_622B_R2', 'LL8_622B_R3', 'LL8_624_r1',
       'LL8_624_r2', 'LL8_624_r3', 'LL8_626_MI_r1', 'LL8_626_MI_r2',
       'LL8_626_MI_r3', 'LL8_631_a_r1', 'LL8_631_a_r2', 'LL8_631_a_r3',
       'LL8_631_b_r1', 'LL8_631_b_r2', 'LL8_631_b_r3'], dtype=object)

In [23]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [30]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [32]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [15]:
df_combo.to_excel('Calibration_fitting.xlsx')